In [ ]:
!pip install matplotlib

In [ ]:
import syft as sy
%matplotlib inline
duet = sy.join_duet(loopback=True)

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.lin =self.torch_ref.nn.Linear(6,10)
    def forward(self, x):
        x = self.lin(x)
        return x

In [ ]:
import torch
model1 = SyNet(torch)
model1_ptr = model1.send(duet)

In [ ]:
opt1 = duet.torch.optim.SGD(params=model1_ptr.parameters(),lr=0.00001)

In [ ]:
duet.store.pandas

In [ ]:
data_pointer = duet.store[0]

In [ ]:
import numpy as np

In [ ]:

model2 = torch.nn.Linear(10,6)
opt2 = torch.optim.SGD(params=model2.parameters(), lr=0.00001)

target = torch.tensor([[19.98,20.03,20.76,21.07,21.11,21.24],[21.35,21.48,21.57,21.63,21.79,21.81],
                       [21.92,22.02,22.19,22.25,22.37,22.48],[22.56,22.61,22.74,22.89,22.93,23.07],
                       [23.16,23.28,23.38,23.46,23.51,23.62],[23.75,23.81,23.94,24.09,24.17,24.28],
                       [24.36,24.41,24.53,24.67,24.79,24.82],[24.94,25.06,25.17,25.28,25.34,25.46],
                       [25.51,25.63,25.79,25.91,26.06,26.13],[26.25,26.37,26.41,26.58,26.69,26.75]])
target.shape
#target.type

In [ ]:
%%time
import matplotlib.pyplot as plt
#epoch = 200
losses = []
predictions = []
for iter in range(250):
    opt1.zero_grad()
    opt2.zero_grad()
    
    activation_ptr = model1_ptr(data_pointer)
    activation = activation_ptr.clone().get(request_block=True)
    
    pred = model2(activation)
    
    #target = target.detach().numpy()
    loss = ((pred - target)**2).sum()
   
    loss.backward()
    
    grad_ptr = activation.grad.clone().send(duet)
    activation_ptr.backward(grad_ptr)
    
    opt1.step()
    opt2.step()
    losses.append(loss.tolist())
    predictions.append(pred.tolist())
    
    print("losses: ",losses)
    print('Predictions =',pred)


In [ ]:
%%time
import matplotlib.pyplot as plt
#%matplotlib inline
#pred = pred.detach().numpy()
#pred = pred.numpy()
#loss = loss.detach().numpy()
# print("lost",losses)
# print("list",losses[0].tolist())
# print("pred", predictions[0].tolist())
# print("pred", predictions[0].ravel().tolist())
#model2 = model2.detach().numpy()
#target = target.detach().numpy()
plt.title('Diameter of Bulb-NFT Hydroponics')
plt.plot(losses,color='r')
plt.ylabel("Loss")
plt.xlabel("epoch")
plt.show()